In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import mnist
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.profiler import model_analyzer
from tensorflow.python.profiler import option_builder

In [2]:
class FLAGS:
  def __init__(self):
    self.hidden1_dim =128     #hidden1 layer 的维度
    self.hidden2_dim = 32     #hidden2 layer 的维度
    self.learning_rate = 1e-2
    self.batch_size = 100
    self.max_step = 100        # 最大的训练步骤
    self.stats_per_steps = 10  # 每隔10步搜集一次RunMetadata。 可以根据需要修改
    self.data_set_dir =  '' #替换为你的本地mnist数据集路径，如果你已经下载过


TRAINING_FLAGS = FLAGS()

In [3]:
data_sets = input_data.read_data_sets(train_dir=TRAINING_FLAGS.data_set_dir,fake_data=False)
images_placeholder = tf.placeholder(tf.float32, shape=(TRAINING_FLAGS.batch_size,
                                                       mnist.IMAGE_PIXELS))
labels_placeholder = tf.placeholder(tf.int32, shape=(TRAINING_FLAGS.batch_size))

logits = mnist.inference(images_placeholder,
                         TRAINING_FLAGS.hidden1_dim,
                         TRAINING_FLAGS.hidden2_dim)
loss = mnist.loss(logits, labels_placeholder)
train_op = mnist.training(loss, TRAINING_FLAGS.learning_rate)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train-labels-idx1-ubyte.gz
Extracting t10k-images-idx3-ubyte.gz
Extracting t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


创建tfprofiler实例，作为记录、处理和显示数据的主体

In [4]:
mnist_profiler = model_analyzer.Profiler(graph=sess.graph)

10 ops no flops stats due to incomplete shapes.


定义trace level为FULL_TRACE，这样我们才能搜集到包括GPU硬件在内的最全统计数据

In [5]:
run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)

创建RunMetadata， 用于在每次session.Run()时汇总统计数据

In [6]:
run_metadata = tf.RunMetadata()

循环执行session.Run()，搜集统计数据并添加到tfprofiler实例中

In [7]:
feed_dict = dict()
for step in range(TRAINING_FLAGS.max_step):
    images_feed, labels_feed = data_sets.train.next_batch(TRAINING_FLAGS.batch_size, fake_data=False)
    feed_dict = {
                 images_placeholder: images_feed,
                 labels_placeholder: labels_feed,
                }    
    #每 TRAINING_FLAGS.stats_per_steps 步，搜集一下统计数据：
    if step % TRAINING_FLAGS.stats_per_steps == 0:
        _, loss_value = sess.run(fetches=[train_op, loss],feed_dict=feed_dict, options=run_options, run_metadata=run_metadata)

        #将本步搜集的统计数据添加到p;tfprofiler实例中     
        mnist_profiler.add_step(step=step, run_meta=run_metadata)
    else:
        _, loss_value = sess.run(fetches=[train_op, loss],
                               feed_dict=feed_dict)

例子1：grpah view显示每个graph node运行时间，并输出到timeline

In [8]:
# 统计内容为每个graph node的运行时间和占用内存
profile_graph_opts_builder = option_builder.ProfileOptionBuilder(
    option_builder.ProfileOptionBuilder.time_and_memory())

# 输出方式为timeline
profile_graph_opts_builder.with_timeline_output(timeline_file='./mnist_profile.json')
# 定义显示sess.Run()第70步的统计数据
profile_graph_opts_builder.with_step(70)

# 显示试图为graph view
mnist_profiler.profile_graph(profile_graph_opts_builder.build())

name: "_TFProfRoot"
total_exec_micros: 552
total_requested_bytes: 983300
total_parameters: 104939
children {
  name: "GradientDescent"
  exec_micros: 3
  total_exec_micros: 537
  total_requested_bytes: 982788
  total_parameters: 104939
  devices: "/job:localhost/replica:0/task:0/device:cpu:0"
  children {
    name: "GradientDescent/value"
    exec_micros: 2
    requested_bytes: 4
    total_exec_micros: 530
    total_requested_bytes: 982532
    total_parameters: 104938
    devices: "/job:localhost/replica:0/task:0/device:cpu:0"
    children {
      name: "GradientDescent/update"
      exec_micros: 1
      total_exec_micros: 528
      total_requested_bytes: 982528
      total_parameters: 104938
      devices: "/job:localhost/replica:0/task:0/device:gpu:0"
      children {
        name: "GradientDescent/update_hidden1/biases/ApplyGradientDescent"
        exec_micros: 11
        total_exec_micros: 490
        total_requested_bytes: 581120
        total_parameters: 104938
        devices: "

例子2：scope view显示模型中的参数数量分布 

In [10]:
# 统计内容为所有trainable Variable Op
profile_scope_opt_builder = option_builder.ProfileOptionBuilder(
    option_builder.ProfileOptionBuilder.trainable_variables_parameter())

# 显示的嵌套深度为4
profile_scope_opt_builder.with_max_depth(4)
# 显示的字段为params，即参数
profile_scope_opt_builder.select(['params'])
# 根据params数量进行显示结果排序
profile_scope_opt_builder.order_by('params')

# 显示试图为scope view
mnist_profiler.profile_name_scope(profile_scope_opt_builder.build())

name: "_TFProfRoot"
total_exec_micros: 11
total_requested_bytes: 420096
total_parameters: 104938
children {
  name: "hidden1"
  total_exec_micros: 6
  total_requested_bytes: 401920
  total_parameters: 100480
  children {
    name: "hidden1/weights"
    exec_micros: 4
    requested_bytes: 401408
    parameters: 100352
    total_exec_micros: 4
    total_requested_bytes: 401408
    total_parameters: 100352
    devices: "/job:localhost/replica:0/task:0/device:gpu:0"
    cpu_exec_micros: 4
    total_cpu_exec_micros: 4
    run_count: 1
    total_run_count: 1
    total_definition_count: 1
    peak_bytes: 401408
    residual_bytes: 401408
    output_bytes: 401408
    total_peak_bytes: 401408
    total_residual_bytes: 401408
    total_output_bytes: 401408
  }
  children {
    name: "hidden1/biases"
    exec_micros: 2
    requested_bytes: 512
    parameters: 128
    total_exec_micros: 2
    total_requested_bytes: 512
    total_parameters: 128
    devices: "/job:localhost/replica:0/task:0/device:

In [ ]:
profile_scope_opt_builder.

例子3： op view显示模型中的expensive node 

In [19]:
profile_op_opt_builder = option_builder.ProfileOptionBuilder()

# 显示字段： op执行时间，使用该op的node数量。 注意：op的执行时间即所有使用该op的node的执行时间总和。
profile_graph_opts_builder.select(['micros', 'occurrence'])

profile_graph_opts_builder.order_by('micros')
profile_op_opt_builder.with_max_depth(4)

# 显示试图为op view
print(mnist_profiler.profile_operations(profile_op_opt_builder.build()))

name: "_TFProfRoot"
total_exec_micros: 603
total_requested_bytes: 983300
total_parameters: 183439
total_float_ops: 63105774
children {
  name: "Add"
  exec_micros: 45
  float_ops: 121768
  total_exec_micros: 603
  total_requested_bytes: 983300
  total_parameters: 183439
  total_float_ops: 63105774
  graph_nodes {
    name: "hidden1/add"
    exec_micros: 20
    total_exec_micros: 20
    devices: "/job:localhost/replica:0/task:0/device:gpu:0"
    float_ops: 12800
    total_float_ops: 12800
    input_shapes {
      value {
        dim {
          size: 100
        }
        dim {
          size: 128
        }
      }
    }
    input_shapes {
      key: 1
      value {
        dim {
          size: 128
        }
      }
    }
    accelerator_exec_micros: 3
    cpu_exec_micros: 16
    total_accelerator_exec_micros: 3
    total_cpu_exec_micros: 16
    run_count: 1
    total_run_count: 1
    total_definition_count: 1
    output_bytes: 51200
    total_output_bytes: 51200
  }
  graph_nodes {
  

In [23]:
_ = tf.profiler.profile(tf.get_default_graph(),
                        run_meta=run_metadata,
                        cmd='scope',
                        options=profile_op_opt_builder.build())

In [18]:
print(mnist_profiler.advise(options=model_analyzer.ALL_ADVICE))

checkers {
  key: "AcceleratorUtilizationChecker"
  value {
    reports: "device: /job:localhost/replica:0/task:0/device:gpu:0 low utilization: 0.15"
  }
}
checkers {
  key: "ExpensiveOperationChecker"
  value {
    reports: "top 1 operation type: MatMul, cpu: 123us, accelerator: 45us, total: 171us (28.36%)\ntop 2 operation type: Sum, cpu: 71us, accelerator: 14us, total: 86us (14.26%)\ntop 3 operation type: ApplyGradientDescent, cpu: 63us, accelerator: 8us, total: 74us (12.27%)"
    reports: "top 1 graph node: gradients, cpu: 0us, accelerator: 0us, total: 0us\ntop 2 graph node: sparse_softmax_cross_entropy_loss, cpu: 0us, accelerator: 0us, total: 0us"
    reports: ""
  }
}
checkers {
  key: "OperationChecker"
  value {
  }
}

